<a href="https://colab.research.google.com/github/cfoli/Multi-label-Medical-Image-Classification/blob/main/Multi_label_CNN_chestXray_classification_6label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import libraries

---



In [ ]:
!pip install lightning torchmetrics

In [24]:
import torch
import torch.nn.functional as F
# import torchvision.datasets as datasets
import torchvision
from torchvision import transforms, models, datasets
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split
import pytorch_lightning as torch_light
from pytorch_lightning.callbacks import EarlyStopping
import torchmetrics
from torchmetrics import Metric
import os
import shutil
import subprocess
import pandas as pd


### Define helper functions and containers

---



In [30]:
BASE_DIR = os.path.join(os.getcwd(), "datasets")

configs = {
    # Image & label properties
    "IMAGE_SIZE":   (224, 224),          # Resize images to (W, H)
    "NUM_CHANNELS": 3,                     # RGB images
    "NUM_CLASSES":  6,                 # Number of output labels
    # "VALID_PCT":    0.1,                  # Fraction of training data for validation

    "BATCH_SIZE": 32,

    # ImageNet dataset normalization values (for pretrained backbones)
    "MEAN": (0.485, 0.456, 0.406),
    "STD":  (0.229, 0.224, 0.225),

    # Paths to download datasets to
    "TRAIN_DATA_DIR": os.path.join(BASE_DIR, "chest-xray", "train"),
    "TEST_DATA_DIR":  os.path.join(BASE_DIR, "chest-xray", "test"),
    "VAL_DATA_DIR":   os.path.join(BASE_DIR, "chest-xray", "val")}

def get_pretrained_model(pretrained_model_name: str, num_classes: int, freeze_backbone: bool = True):
    """
    Load a pretrained Torchvision classification model and replace
    ONLY its final Linear layer for transfer learning or fine-tuning.
    """

    # Load pretrained model from torchvision
    model = getattr(torchvision.models, pretrained_model_name)(weights="DEFAULT")

    # Optionally freeze all pretrained parameters (the backbone)
    if freeze_backbone:
        for param in model.parameters():
            param.requires_grad = False

    # Get the last top-level module (typically the classifier head) from the pretrained model
    layer_modules = list(model.named_children())
    last_module_name, last_module = layer_modules[-1]

    if isinstance(last_module, nn.Sequential):
        # If the classifier is a Sequential module, replace its final layer

        num_in_features = last_module[-1].in_features # find the dimensionality of the input to the last layer of the network
        last_module[-1] = nn.Linear(num_in_features, num_classes)

    else:
        # Otherwise, replace the module directly (e.g., ResNet-style fc layer)
        in_features = last_module.in_features
        setattr(model, last_module_name, nn.Linear(in_features, num_classes))

    return model


### Load kaggle credentials (i.e., API) file

---



In [32]:
# Import kaggle API token file
from google.colab import files
files.upload() # This will prompt you to select and upload the file

# Then move and secure the file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


### Create model (i.e. classifier) module

---



In [28]:
class modelModule(torch_light.LightningModule):
    def __init__(self, num_classes, backbone_model_name):
        super().__init__()
        self.num_classes = num_classes
        self.backbone_model_name = backbone_model_name

        # Load a pretrained backbone and replace its final layer
        self.model = get_pretrained_model(
            num_classes = self.num_classes,
            pretrained_model_name = self.backbone_model_name )

        # Binary classification loss operating on raw logits
        self.loss_function = torch.nn.BCEWithLogitsLoss()
        self.accuracy_function = torchmetrics.Accuracy(task="multilabel", num_labels=self.num_classes)
        self.f1_score_function = torchmetrics.F1Score(task="multilabel", num_labels=self.num_classes)

    def forward(self, x):
        # Forward pass through the backbone model
        return self.model(x)

    def _common_step(self, batch, batch_idx):
        """
        Shared logic for train / val / test steps.
        Computes loss and evaluation metrics.
        """
        x, y = batch

        # Compute model predictions ()
        y_hat = self.forward(x)

        # Compute metrics (expects logits + labels)
        loss     = self.loss_function(y_hat, y)
        accuracy = self.accuracy_function(y_hat, y)
        f1_score = self.f1_score_function(y_hat, y)

        return loss, y_hat, y, accuracy, f1_score

    def training_step(self, batch, batch_idx):
        # Run shared step
        loss, y_hat, y, accuracy, f1_score = self._common_step(batch, batch_idx)

        # Log epoch-level training metrics
        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy, "train_f1_score": f1_score},
            on_step=False, on_epoch=True, prog_bar=True)

        # Lightning expects the loss key for backprop
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        # Run shared step
        loss, y_hat, y, accuracy, f1_score = self._common_step(batch, batch_idx)

        # Log validation metrics
        self.log_dict(
            {"val_loss": loss, "val_accuracy": accuracy,"val_f1_score": f1_score},
            on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # Run shared step
        loss, y_hat, y, accuracy, f1_score = self._common_step(batch, batch_idx)

        # Log test metrics
        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy,"test_f1_score": f1_score},
            on_step=False, on_epoch=True, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        """
        Prediction logic used by trainer.predict().
        Returns model outputs without computing loss.
        """
        x = batch if not isinstance(batch, (tuple, list)) else batch[0]
        logits = self.forward(x)

        # Convert logits to probabilities for inference
        probs = torch.sigmoid(logits)

        return probs

    def configure_optimizers(self):
        # Optimizer over all trainable parameters
        optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer


### **Create data module**

In [34]:
class dataModule(torch_light.LightningDataModule):
  def __init__(self, batch_size, num_classes):
    super().__init__()

    self.num_classes = num_classes
    # self.validation_size = validation_size # validation_size = 0.1
    self.batch_size = batch_size

    self.train_transforms = transforms.Compose([
                transforms.RandomAffine(degrees=40, translate=(0.01, 0.12), shear=0.05),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ToTensor(),
                # transforms.Normalize(dataConfig["MEAN"], dataConfig["STD"], inplace=True),
                transforms.RandomErasing(inplace=True)])

    self.valid_transforms = transforms.Compose([
                transforms.ToTensor(),
                # transforms.Normalize(dataConfig["MEAN"], dataConfig["STD"], inplace=True)
                ])
    self.test_transforms = self.valid_transforms

  def prepare_data(self):
      # Download datasets from Kaggle if not already downloaded
      if os.path.exists(BASE_DIR):
          return  # already downloaded

      # Kaggle expects the API key (i.e., json) to live in a directory like ~/.kaggle/kaggle.json. So we create that directory and copy the loaded Kaggle file there
      kaggle_dir  = os.path.join(os.path.expanduser("~"), ".kaggle")
      kaggle_json = os.path.join(kaggle_dir, "kaggle.json")

      if not os.path.exists(kaggle_json):
          os.makedirs(kaggle_dir, exist_ok=True)
          shutil.copyfile("kaggle.json", kaggle_json)
          os.chmod(kaggle_json, 0o600)

      print("Downloading dataset from Kaggle...")

      # Download datasets from Kaggle
      subprocess.run(
          ["kaggle", "datasets", "download","-q", "mohamedasak/chest-x-ray-6-classes-dataset", "-p", BASE_DIR, "--unzip"],
          check=True)
      print("Download complete!")

  def setup(self, stage=None):

    self.train_dataset = datasets.ImageFolder(
        root=configs["TRAIN_DATA_DIR"],
        transform=self.train_transforms)

    self.val_dataset = datasets.ImageFolder(
        root=configs["VAL_DATA_DIR"],
        transform=self.valid_transforms)

    self.test_dataset = datasets.ImageFolder(
        root=configs["TEST_DATA_DIR"],
        transform=self.test_transforms)

  def train_dataloader(self):
    # create training data loader object from training dataset
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)


In [33]:
model_ = modelModule(
    num_classes = configs["NUM_CLASSES"],
    backbone_model_name = "alexnet")
    # efficientnet_v2_s, convnext_small, alexnet, resnet50, vgg11

data_module = dataModule(
    batch_size      = configs["BATCH_SIZE"],
    num_classes     = configs["NUM_CLASSES"])

trainer_ = torch_light.Trainer(
    accelerator = "auto",
    devices     = "auto",
    strategy    = "auto",
    max_epochs  = 10,
    callbacks   = [EarlyStopping(patience=5, verbose=False, monitor="val_loss")],
    precision   = "16")

trainer_.fit(model_, data_module)
trainer_.validate(model_, data_module)
trainer_.test(model_, data_module)


INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


Download complete!


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/model_summary/model_summary.py:242: Precision bf16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.


┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name              ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model             │ AlexNet            │ 57.0 M │ train │     0 │
│ 1 │ loss_function     │ BCEWithLogitsLoss  │      0 │ train │     0 │
│ 2 │ accuracy_function │ MultilabelAccuracy │      0 │ train │     0 │
│ 3 │ f1_score_function │ MultilabelF1Score  │      0 │ train │     0 │
└───┴───────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 24.6 K                                                                                           
Non-trainable params: 57.0 M                                                                                       
Total params: 57.0 M                                                                                               
Total estimated model params size (MB): 228                                                                        
Modules in train mode: 27                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:485: Your 
`val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for 
val/test dataloaders.

ValueError: Target size (torch.Size([32])) must be the same as input size (torch.Size([32, 6]))